In [454]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [455]:
df = pd.read_csv("API_GHA_DS2_en_csv_v2_23038.csv", skiprows=4)

In [456]:
indices_to_drop = [0, 1, 3] + list(range(4, 47)) + [-1]
df.drop(df.columns[indices_to_drop], axis=1, inplace=True)
# df.drop(df.columns[4:47], axis=1, inplace=True)
# df.drop(df.columns[[1,3,-1]], axis=1, inplace=True)

In [457]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1496 entries, 0 to 1495
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Indicator Name  1496 non-null   object 
 1   2003            951 non-null    float64
 2   2004            872 non-null    float64
 3   2005            980 non-null    float64
 4   2006            1069 non-null   float64
 5   2007            1054 non-null   float64
 6   2008            1085 non-null   float64
 7   2009            1038 non-null   float64
 8   2010            1068 non-null   float64
 9   2011            1050 non-null   float64
 10  2012            1091 non-null   float64
 11  2013            1138 non-null   float64
 12  2014            1089 non-null   float64
 13  2015            1094 non-null   float64
 14  2016            1093 non-null   float64
 15  2017            1150 non-null   float64
 16  2018            1076 non-null   float64
 17  2019            1032 non-null   f

In [458]:
df.head()

,Indicator Name,2003,2004,2005,2006,2007,2008,2009,2010,2011,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,"Intentional homicides, female (per 100,000 fem...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.468212,0.577005,NaN,NaN
1,Battle-related deaths (number of people),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Voice and Accountability: Percentile Rank,57.213932,54.326923,57.211540,58.653847,60.576923,58.653847,61.611374,63.033176,62.441315,...,61.576355,64.039406,67.487686,66.995071,67.475731,64.734299,63.285023,60.869564,59.420288,NaN
3,Transport services (% of commercial service ex...,20.293481,20.011298,13.473930,16.362964,19.401695,15.366598,21.206738,27.071397,29.379012,...,31.152559,7.606824,12.925820,7.303608,5.990399,5.076937,6.811068,6.237314,6.573509,4.996396
4,"Computer, communications and other services (%...",10.906329,10.706059,8.571011,13.568551,23.560711,24.836941,27.393858,25.505913,27.591708,...,21.730904,78.323697,72.497534,79.001065,81.006457,80.063366,80.233787,70.325428,65.980425,82.722004


In [459]:
df['Indicator Name'].value_counts()

Indicator Name
Intentional homicides, female (per 100,000 female)                                                                                 1
Unemployment, male (% of male labor force) (modeled ILO estimate)                                                                  1
Prevalence of underweight, weight for age (% of children under 5)                                                                  1
People practicing open defecation, urban (% of urban population)                                                                   1
Prevalence of overweight, weight for height, female (% of children under 5)                                                        1
                                                                                                                                  ..
Risk premium on lending (lending rate minus treasury bill rate, %)                                                                 1
Domestic credit to private sector (% of GDP)          

In [460]:
df.head()

,Indicator Name,2003,2004,2005,2006,2007,2008,2009,2010,2011,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,"Intentional homicides, female (per 100,000 fem...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.468212,0.577005,NaN,NaN
1,Battle-related deaths (number of people),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Voice and Accountability: Percentile Rank,57.213932,54.326923,57.211540,58.653847,60.576923,58.653847,61.611374,63.033176,62.441315,...,61.576355,64.039406,67.487686,66.995071,67.475731,64.734299,63.285023,60.869564,59.420288,NaN
3,Transport services (% of commercial service ex...,20.293481,20.011298,13.473930,16.362964,19.401695,15.366598,21.206738,27.071397,29.379012,...,31.152559,7.606824,12.925820,7.303608,5.990399,5.076937,6.811068,6.237314,6.573509,4.996396
4,"Computer, communications and other services (%...",10.906329,10.706059,8.571011,13.568551,23.560711,24.836941,27.393858,25.505913,27.591708,...,21.730904,78.323697,72.497534,79.001065,81.006457,80.063366,80.233787,70.325428,65.980425,82.722004


In [461]:
def filter_df(df: pd.DataFrame, colname:str, keywords:list =[], case:bool =False) -> pd.DataFrame:
    keywords = []+keywords
    pattern = '|'.join(map(re.escape, keywords))
    df[colname] = df[colname].astype(str).str.strip()
    df = df[df[colname].str.contains(pattern, case)]
    return df

In [462]:
# ['GDP', 'employment', 'government revenue']
# df = filter_df(df, 'Indicator Name', ['GDP', 'employment', 'government revenue'])

In [463]:
# df.info()

In [464]:
# df.head()

In [465]:
df = filter_df(
    df,
    'Indicator Name',
    [
        'GDP (constant 2015 US$)',
        'Unemployment, total (% of total labor force) (modeled ILO estimate)',
        'Total natural resources rents (% of GDP)' 
    ],
    case=False
)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3 entries, 76 to 1353
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Indicator Name  3 non-null      object 
 1   2003            3 non-null      float64
 2   2004            3 non-null      float64
 3   2005            3 non-null      float64
 4   2006            3 non-null      float64
 5   2007            3 non-null      float64
 6   2008            3 non-null      float64
 7   2009            3 non-null      float64
 8   2010            3 non-null      float64
 9   2011            3 non-null      float64
 10  2012            3 non-null      float64
 11  2013            3 non-null      float64
 12  2014            3 non-null      float64
 13  2015            3 non-null      float64
 14  2016            3 non-null      float64
 15  2017            3 non-null      float64
 16  2018            3 non-null      float64
 17  2019            3 non-null      float64


In [466]:
df.head()

,Indicator Name,2003,2004,2005,2006,2007,2008,2009,2010,2011,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
76,Total natural resources rents (% of GDP),1.561977e+01,1.239741e+01,1.173180e+01,7.576185e+00,8.828153e+00,9.161902e+00,1.081158e+01,1.011463e+01,1.607069e+01,...,1.224953e+01,1.056352e+01,8.649803e+00,1.037004e+01,9.790408e+00,9.767367e+00,8.486580e+00,1.334748e+01,NaN,NaN
641,GDP (constant 2015 US$),2.293716e+10,2.422164e+10,2.565072e+10,2.729234e+10,2.847869e+10,3.108443e+10,3.259031e+10,3.516486e+10,4.010451e+10,...,4.837999e+10,4.940601e+10,5.107270e+10,5.522435e+10,5.864830e+10,6.246500e+10,6.278604e+10,6.597335e+10,6.849189e+10,7.050882e+10
1353,"Unemployment, total (% of total labor force) (...",7.722000e+00,6.757000e+00,5.817000e+00,4.897000e+00,5.092000e+00,5.142000e+00,5.349000e+00,5.384000e+00,4.227000e+00,...,4.412000e+00,6.806000e+00,5.237000e+00,3.369000e+00,3.251000e+00,3.156000e+00,3.285000e+00,3.338000e+00,3.084000e+00,3.079000e+00


In [467]:
numeric_cols = df.select_dtypes(include=[np.number])
df[numeric_cols.columns] = numeric_cols.apply(lambda row: row.fillna(row.mean()), axis=1)
df.head()

,Indicator Name,2003,2004,2005,2006,2007,2008,2009,2010,2011,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
76,Total natural resources rents (% of GDP),1.561977e+01,1.239741e+01,1.173180e+01,7.576185e+00,8.828153e+00,9.161902e+00,1.081158e+01,1.011463e+01,1.607069e+01,...,1.224953e+01,1.056352e+01,8.649803e+00,1.037004e+01,9.790408e+00,9.767367e+00,8.486580e+00,1.334748e+01,1.117099e+01,1.117099e+01
641,GDP (constant 2015 US$),2.293716e+10,2.422164e+10,2.565072e+10,2.729234e+10,2.847869e+10,3.108443e+10,3.259031e+10,3.516486e+10,4.010451e+10,...,4.837999e+10,4.940601e+10,5.107270e+10,5.522435e+10,5.864830e+10,6.246500e+10,6.278604e+10,6.597335e+10,6.849189e+10,7.050882e+10
1353,"Unemployment, total (% of total labor force) (...",7.722000e+00,6.757000e+00,5.817000e+00,4.897000e+00,5.092000e+00,5.142000e+00,5.349000e+00,5.384000e+00,4.227000e+00,...,4.412000e+00,6.806000e+00,5.237000e+00,3.369000e+00,3.251000e+00,3.156000e+00,3.285000e+00,3.338000e+00,3.084000e+00,3.079000e+00


In [468]:
indicator_df = df

In [469]:
file_path = 'Commodity Prices Monthly.csv'
df_csv = pd.read_csv(file_path)

# Ensure the 'Dec' column is numeric (clean commas or other non-numeric characters)
df_csv['Dec'] = df_csv['Dec'].replace(',', '', regex=True).astype(float)

# Filter December values and pivot the DataFrame
df_december = df_csv[['Year', 'Variables', 'Dec']].copy()
df_december = df_december.loc[df_december['Variables'].isin([
    'International Brent Crude Oil Price (US$/Barrel) - Monthly Average',
    'International Cocoa Price (US$/Tonne) - Monthly Average',
    'International Gold Price (US$/fine ounce) - Monthly Average'
])]
df_december_final = df_december.pivot_table(
    index='Variables',    # Indicator Name
    columns='Year',       # Years
    values='Dec',         # December values
    aggfunc='mean'
).reset_index()

# Rename 'Variables' to 'Indicator Name'
df_december_final.rename(columns={'Variables': 'Indicator Name'},inplace=True)

df_december_final['Indicator Name'] = df_december_final['Indicator Name'].replace({
    'International Brent Crude Oil Price (US$/Barrel) - Monthly Average' : 'Brent Crude Oil Price (US$/Barrel)',
    'International Cocoa Price (US$/Tonne) - Monthly Average' : 'Cocoa Price (US$/Tonne)',
    'International Gold Price (US$/fine ounce) - Monthly Average' : 'Gold Price (US$/fine ounce)'
})

In [470]:
price_df = df_december_final

In [471]:
indicator_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3 entries, 76 to 1353
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Indicator Name  3 non-null      object 
 1   2003            3 non-null      float64
 2   2004            3 non-null      float64
 3   2005            3 non-null      float64
 4   2006            3 non-null      float64
 5   2007            3 non-null      float64
 6   2008            3 non-null      float64
 7   2009            3 non-null      float64
 8   2010            3 non-null      float64
 9   2011            3 non-null      float64
 10  2012            3 non-null      float64
 11  2013            3 non-null      float64
 12  2014            3 non-null      float64
 13  2015            3 non-null      float64
 14  2016            3 non-null      float64
 15  2017            3 non-null      float64
 16  2018            3 non-null      float64
 17  2019            3 non-null      float64


In [472]:
price_df['Indicator Name'].unique()

array(['Brent Crude Oil Price (US$/Barrel)', 'Cocoa Price (US$/Tonne)',
       'Gold Price (US$/fine ounce)'], dtype=object)

In [473]:
price_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Indicator Name  3 non-null      object 
 1   2000            3 non-null      float64
 2   2001            2 non-null      float64
 3   2002            3 non-null      float64
 4   2003            3 non-null      float64
 5   2004            3 non-null      float64
 6   2005            3 non-null      float64
 7   2006            3 non-null      float64
 8   2007            3 non-null      float64
 9   2008            3 non-null      float64
 10  2009            3 non-null      float64
 11  2010            3 non-null      float64
 12  2011            3 non-null      float64
 13  2012            3 non-null      float64
 14  2013            3 non-null      float64
 15  2014            3 non-null      float64
 16  2015            3 non-null      float64
 17  2016            3 non-null      float64

In [474]:
price_df.head()

Year,Indicator Name,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Brent Crude Oil Price (US$/Barrel),26.67,NaN,27.45,29.58,40.64,57.61,62.54,91.51,43.29,...,62.36,38.92,54.93,64.27,57.67,65.17,50.23,74.80,81.34,0.0
1,Cocoa Price (US$/Tonne),717.94,1290.15,2013.65,1590.14,1620.10,1461.57,1629.26,2071.90,2445.10,...,2919.09,3300.76,2268.36,1904.60,2256.25,2518.38,2581.32,2481.95,2538.57,0.0
2,Gold Price (US$/fine ounce),271.33,276.29,333.37,408.00,441.85,510.91,628.71,807.20,821.49,...,1200.68,1069.40,1151.16,1266.56,1251.11,1481.33,1857.19,1790.90,1796.21,0.0


replacing missing data with 'Annual Report' values found on Bank of Ghana website

In [475]:
indicator_df.columns = [str(col) for col in indicator_df.columns]
price_df.columns = [str(col) for col in price_df.columns]

# Replace NaN with 24.45 for '2001'
if pd.notna(price_df.loc[0, '2001']):
    price_df.loc[0, '2001'] = price_df.loc[0, '2001']
else:
    price_df.loc[0, '2001'] = 24.45

# Replace 0 with 82.19 for the cell at row 0, column '2023'
if price_df.loc[0, '2023'] == 0:
    price_df.loc[0, '2023'] = 82.19

# Replace 0 with 4235.60 for the cell at row 1, column '2023'
if price_df.loc[1, '2023'] == 0:
    price_df.loc[1, '2023'] = 4235.60

# Replace 0 with 2035.43 for the cell at row 2, column '2023'
if price_df.loc[2, '2023'] == 0:
    price_df.loc[2, '2023'] = 2035.43

In [476]:
price_df.head()

,Indicator Name,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Brent Crude Oil Price (US$/Barrel),26.67,24.45,27.45,29.58,40.64,57.61,62.54,91.51,43.29,...,62.36,38.92,54.93,64.27,57.67,65.17,50.23,74.80,81.34,82.19
1,Cocoa Price (US$/Tonne),717.94,1290.15,2013.65,1590.14,1620.10,1461.57,1629.26,2071.90,2445.10,...,2919.09,3300.76,2268.36,1904.60,2256.25,2518.38,2581.32,2481.95,2538.57,4235.60
2,Gold Price (US$/fine ounce),271.33,276.29,333.37,408.00,441.85,510.91,628.71,807.20,821.49,...,1200.68,1069.40,1151.16,1266.56,1251.11,1481.33,1857.19,1790.90,1796.21,2035.43


In [477]:
price_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Indicator Name  3 non-null      object 
 1   2000            3 non-null      float64
 2   2001            3 non-null      float64
 3   2002            3 non-null      float64
 4   2003            3 non-null      float64
 5   2004            3 non-null      float64
 6   2005            3 non-null      float64
 7   2006            3 non-null      float64
 8   2007            3 non-null      float64
 9   2008            3 non-null      float64
 10  2009            3 non-null      float64
 11  2010            3 non-null      float64
 12  2011            3 non-null      float64
 13  2012            3 non-null      float64
 14  2013            3 non-null      float64
 15  2014            3 non-null      float64
 16  2015            3 non-null      float64
 17  2016            3 non-null      float64

In [478]:
main_df = pd.concat([price_df, indicator_df], axis=0)
main_df = main_df.reset_index(drop=True)
main_df

,Indicator Name,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Brent Crude Oil Price (US$/Barrel),26.67,24.45,27.45,2.958000e+01,4.064000e+01,5.761000e+01,6.254000e+01,9.151000e+01,4.329000e+01,...,6.236000e+01,3.892000e+01,5.493000e+01,6.427000e+01,5.767000e+01,6.517000e+01,5.023000e+01,7.480000e+01,8.134000e+01,8.219000e+01
1,Cocoa Price (US$/Tonne),717.94,1290.15,2013.65,1.590140e+03,1.620100e+03,1.461570e+03,1.629260e+03,2.071900e+03,2.445100e+03,...,2.919090e+03,3.300760e+03,2.268360e+03,1.904600e+03,2.256250e+03,2.518380e+03,2.581320e+03,2.481950e+03,2.538570e+03,4.235600e+03
2,Gold Price (US$/fine ounce),271.33,276.29,333.37,4.080000e+02,4.418500e+02,5.109100e+02,6.287100e+02,8.072000e+02,8.214900e+02,...,1.200680e+03,1.069400e+03,1.151160e+03,1.266560e+03,1.251110e+03,1.481330e+03,1.857190e+03,1.790900e+03,1.796210e+03,2.035430e+03
3,Total natural resources rents (% of GDP),NaN,NaN,NaN,1.561977e+01,1.239741e+01,1.173180e+01,7.576185e+00,8.828153e+00,9.161902e+00,...,1.224953e+01,1.056352e+01,8.649803e+00,1.037004e+01,9.790408e+00,9.767367e+00,8.486580e+00,1.334748e+01,1.117099e+01,1.117099e+01
4,GDP (constant 2015 US$),NaN,NaN,NaN,2.293716e+10,2.422164e+10,2.565072e+10,2.729234e+10,2.847869e+10,3.108443e+10,...,4.837999e+10,4.940601e+10,5.107270e+10,5.522435e+10,5.864830e+10,6.246500e+10,6.278604e+10,6.597335e+10,6.849189e+10,7.050882e+10
5,"Unemployment, total (% of total labor force) (...",NaN,NaN,NaN,7.722000e+00,6.757000e+00,5.817000e+00,4.897000e+00,5.092000e+00,5.142000e+00,...,4.412000e+00,6.806000e+00,5.237000e+00,3.369000e+00,3.251000e+00,3.156000e+00,3.285000e+00,3.338000e+00,3.084000e+00,3.079000e+00


In [479]:
main_df['2002'] = main_df.apply(lambda row: row['2003'] if pd.isna(row['2002']) else row['2002'], axis=1)
main_df['2001'] = main_df.apply(lambda row: row['2002'] if pd.isna(row['2001']) else row['2001'], axis=1)
main_df['2000'] = main_df.apply(lambda row: row['2001'] if pd.isna(row['2000']) else row['2000'], axis=1)

In [480]:
main_df

,Indicator Name,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Brent Crude Oil Price (US$/Barrel),2.667000e+01,2.445000e+01,2.745000e+01,2.958000e+01,4.064000e+01,5.761000e+01,6.254000e+01,9.151000e+01,4.329000e+01,...,6.236000e+01,3.892000e+01,5.493000e+01,6.427000e+01,5.767000e+01,6.517000e+01,5.023000e+01,7.480000e+01,8.134000e+01,8.219000e+01
1,Cocoa Price (US$/Tonne),7.179400e+02,1.290150e+03,2.013650e+03,1.590140e+03,1.620100e+03,1.461570e+03,1.629260e+03,2.071900e+03,2.445100e+03,...,2.919090e+03,3.300760e+03,2.268360e+03,1.904600e+03,2.256250e+03,2.518380e+03,2.581320e+03,2.481950e+03,2.538570e+03,4.235600e+03
2,Gold Price (US$/fine ounce),2.713300e+02,2.762900e+02,3.333700e+02,4.080000e+02,4.418500e+02,5.109100e+02,6.287100e+02,8.072000e+02,8.214900e+02,...,1.200680e+03,1.069400e+03,1.151160e+03,1.266560e+03,1.251110e+03,1.481330e+03,1.857190e+03,1.790900e+03,1.796210e+03,2.035430e+03
3,Total natural resources rents (% of GDP),1.561977e+01,1.561977e+01,1.561977e+01,1.561977e+01,1.239741e+01,1.173180e+01,7.576185e+00,8.828153e+00,9.161902e+00,...,1.224953e+01,1.056352e+01,8.649803e+00,1.037004e+01,9.790408e+00,9.767367e+00,8.486580e+00,1.334748e+01,1.117099e+01,1.117099e+01
4,GDP (constant 2015 US$),2.293716e+10,2.293716e+10,2.293716e+10,2.293716e+10,2.422164e+10,2.565072e+10,2.729234e+10,2.847869e+10,3.108443e+10,...,4.837999e+10,4.940601e+10,5.107270e+10,5.522435e+10,5.864830e+10,6.246500e+10,6.278604e+10,6.597335e+10,6.849189e+10,7.050882e+10
5,"Unemployment, total (% of total labor force) (...",7.722000e+00,7.722000e+00,7.722000e+00,7.722000e+00,6.757000e+00,5.817000e+00,4.897000e+00,5.092000e+00,5.142000e+00,...,4.412000e+00,6.806000e+00,5.237000e+00,3.369000e+00,3.251000e+00,3.156000e+00,3.285000e+00,3.338000e+00,3.084000e+00,3.079000e+00


In [481]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Indicator Name  6 non-null      object 
 1   2000            6 non-null      float64
 2   2001            6 non-null      float64
 3   2002            6 non-null      float64
 4   2003            6 non-null      float64
 5   2004            6 non-null      float64
 6   2005            6 non-null      float64
 7   2006            6 non-null      float64
 8   2007            6 non-null      float64
 9   2008            6 non-null      float64
 10  2009            6 non-null      float64
 11  2010            6 non-null      float64
 12  2011            6 non-null      float64
 13  2012            6 non-null      float64
 14  2013            6 non-null      float64
 15  2014            6 non-null      float64
 16  2015            6 non-null      float64
 17  2016            6 non-null      float64

In [482]:
main_df.to_csv("main_data.csv", index=False)